In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Downloads/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
links = pd.read_csv('data/export.csv')
links

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89736,151374,https://www.slowboring.com/p/sunday-thread-84b,Sunday Thread,\N,\N,2022-06-12 16:55:05.163399+00,2022-06-12 16:55:05.163399+00,NaN,\N,Yglesias,15.0,0.0,\N
89737,151375,https://forum.effectivealtruism.org/posts/8bqc...,New cause area: bivalve aquaculture,\N,\N,2022-06-12 16:55:05.164768+00,2022-06-12 16:55:05.164768+00,NaN,\N,EAForum,76.0,0.0,\N
89738,151376,https://stefanfschubert.com/blog/2022/6/12/str...,Deliberate altruism and costly signalling,\N,\N,2022-06-12 16:55:05.166083+00,2022-06-12 16:55:05.166083+00,NaN,\N,Schubert,60.0,0.0,\N
89739,151377,https://frenchpress.thedispatch.com/p/there-is...,There Is No Right Person to Hate,\N,\N,2022-06-12 16:55:05.167382+00,2022-06-12 16:55:05.167382+00,NaN,\N,Dispatch,97.0,0.0,\N


In [2]:
links['added'].max()

'2022-06-12 19:02:47.552048+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
89315    150930
89316    150931
89333    150949
89434    151056
89740    151379
Name: id, Length: 8385, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
89732,151370,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2022-06-12 16:55:05.157385+00,2022-06-12 16:55:05.157385+00,NaN,\N,CEA,35.0,0.0,\N
89740,151379,https://nadia.xyz/idea-machines,Idea Machines,\N,nadia.xyz,2022-06-12 19:02:47.552048+00,2022-06-12 19:02:47.552041+00,NaN,\N,Custom,1.0,0.0,1


In [5]:
links[links['url'].duplicated()]['id']

89732    151370
89740    151379
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Downloads/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/68
...11/68
...21/68
...31/68
...41/68
...51/68
...61/68
...68/68


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89803,151442,https://steady.substack.com/p/guns-abortion-an...,Guns Abortion and the American Flag,\N,\N,2022-06-12 19:32:35.828373,2022-06-12 19:32:35.828385,\N,\N,Custom,3.0,0.0,NaN
89804,151443,https://www.washingtonpost.com/technology/2022...,The Google engineer who thinks the company’s A...,\N,\N,2022-06-12 19:32:35.893295,2022-06-12 19:32:35.893303,\N,\N,Custom,58.0,0.0,NaN
89805,151444,https://ghost.org/resources/types-of-newsletters/,6 types of newsletters you can start today,\N,\N,2022-06-12 19:32:35.943339,2022-06-12 19:32:35.943353,\N,\N,Custom,37.0,0.0,NaN
89806,151445,https://www.niemanlab.org/2022/04/algorithms-l...,Algorithms lies and social media,\N,\N,2022-06-12 19:32:35.991399,2022-06-12 19:32:35.991413,\N,\N,Custom,42.0,0.0,NaN


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89803,151442,https://steady.substack.com/p/guns-abortion-an...,Guns Abortion and the American Flag,\N,\N,2022-06-12 19:32:35.828373,2022-06-12 19:32:35.828385,\N,\N,Custom,3.0,0.0,NaN
89804,151443,https://www.washingtonpost.com/technology/2022...,The Google engineer who thinks the company’s A...,\N,\N,2022-06-12 19:32:35.893295,2022-06-12 19:32:35.893303,\N,\N,Custom,58.0,0.0,NaN
89805,151444,https://ghost.org/resources/types-of-newsletters/,6 types of newsletters you can start today,\N,\N,2022-06-12 19:32:35.943339,2022-06-12 19:32:35.943353,\N,\N,Custom,37.0,0.0,NaN
89806,151445,https://www.niemanlab.org/2022/04/algorithms-l...,Algorithms lies and social media,\N,\N,2022-06-12 19:32:35.991399,2022-06-12 19:32:35.991413,\N,\N,Custom,42.0,0.0,NaN


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89803,151442,https://steady.substack.com/p/guns-abortion-an...,Guns Abortion and the American Flag,\N,\N,2022-06-12 19:32:35.828373,2022-06-12 19:32:35.828385,\N,\N,Custom,3,0,NaN
89804,151443,https://www.washingtonpost.com/technology/2022...,The Google engineer who thinks the company’s A...,\N,\N,2022-06-12 19:32:35.893295,2022-06-12 19:32:35.893303,\N,\N,Custom,58,0,NaN
89805,151444,https://ghost.org/resources/types-of-newsletters/,6 types of newsletters you can start today,\N,\N,2022-06-12 19:32:35.943339,2022-06-12 19:32:35.943353,\N,\N,Custom,37,0,NaN
89806,151445,https://www.niemanlab.org/2022/04/algorithms-l...,Algorithms lies and social media,\N,\N,2022-06-12 19:32:35.991399,2022-06-12 19:32:35.991413,\N,\N,Custom,42,0,NaN


In [10]:
links.to_csv('data/export.csv', index=False)